In [1]:
import torch
import pandas as pd
import torch.optim as optim
import requests
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = Parameter(torch.ones(24) * 0.98) 
        self.w2 = Parameter(torch.ones(24) * 1.18)

    def forward(self, oracle_demand, oracle_solar, oracle_price):
        bid_demand = (oracle_demand - oracle_solar) * self.w1
        bid_price = oracle_price * self.w2
        bid_price.data.clamp_(max=7)
        return bid_demand, bid_price

In [3]:
demand_train_oracle = pd.read_csv('./Data/Demand_Train_pred.csv', header=None)
demand_train_actual = pd.read_csv('./Data/Demand_Train.csv', header=None)
solar_train_oracle = pd.read_csv('./Data/Solar_Train_pred.csv', header=None)
solar_train_actual = pd.read_csv('./Data/Solar_Train.csv', header=None)
price_train_oracle = pd.read_csv('./Data/Price_Train_pred.csv', header=None)
price_train_actual = pd.read_csv('./Data/Price_Train.csv', header=None)

In [4]:
def cost(bid_quantities, bid_prices, demands, solar_outputs, prices):
    BATTERY_CAPACITY = 25
    BATTERY_CAPACITY_PER_HOUR = 5
    BATTERY_EFFICIENCY = 0.8
    DISCOM_RATE = 7
    battery = Variable(torch.Tensor([0]))
    cost = Variable(torch.Tensor([0]))
    result = []
    for bid_quantity, bid_price, demand, solar_output, price in zip(bid_quantities.contiguous().view(-1), bid_prices.contiguous().view(-1), demands.contiguous().view(-1), solar_outputs.contiguous().view(-1), prices.contiguous().view(-1)):
        energy_from_sun = torch.min(solar_output, demand)
        bid_won = bid_price > price
        energy_from_market = bid_won.float() * bid_quantity
        energy_from_battery = torch.min(torch.clamp((demand - energy_from_sun - energy_from_market) / BATTERY_EFFICIENCY, min=0),
                                        torch.clamp(battery, max=BATTERY_CAPACITY_PER_HOUR)) * BATTERY_EFFICIENCY
        energy_from_DISCOM = torch.clamp(demand - energy_from_sun - energy_from_market - energy_from_battery, min=0)
        
        residual_energy = torch.clamp(energy_from_market + energy_from_sun - demand, min=0)
        battery = battery + torch.clamp(residual_energy, max=BATTERY_CAPACITY_PER_HOUR) - energy_from_battery / BATTERY_EFFICIENCY
        battery = torch.clamp(battery, min=0, max=BATTERY_CAPACITY)
        
        cost += energy_from_market * bid_price + energy_from_DISCOM * DISCOM_RATE
    return cost

In [5]:
net = Net()

In [6]:
data_size = demand_train_actual.values.shape[0]
epochs = 100
batch_size = 5
learning_rate = 10 ** (-10)

In [7]:
for _ in range(epochs):
    start = 0
    end = batch_size
    epoch_loss = 0
    while end <= data_size:
        net.zero_grad()
        bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(solar_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(price_train_oracle.values[start:end])))
        loss = cost(bid_quantities, bid_prices,
             Variable(torch.Tensor(demand_train_actual.values[start:end])),
             Variable(torch.Tensor(solar_train_actual.values[start:end])),
             Variable(torch.Tensor(price_train_actual.values[start:end])))
        loss.backward()
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate)
            f.data.clamp_(min=0.5, max=2)
        epoch_loss += loss.data[0]
        
        start += batch_size
        end += batch_size
    print(epoch_loss * batch_size / data_size)

31957.687174479168
31957.019520399306
31956.352235243055
31955.685199652777
31955.017740885418
31954.350944010417
31953.683919270832
31953.01675347222
31952.34997829861
31951.683235677083
31951.015418836807
31950.349576822915
31949.682964409723
31949.016200086804
31948.349370659722
31947.682107204862
31947.01589626736
31946.349088541665
31945.6818359375
31945.015158420138
31944.350499131946
31943.683246527777
31943.0166015625
31942.350238715277
31941.684592013888
31941.017296006943
31940.352289496528
31939.686512586806
31939.01986762153
31938.353797743057
31937.687337239582
31937.020323350695
31936.354676649305
31935.6875
31935.021744791666
31934.35615234375
31933.689995659723
31933.023046875
31932.357074652777
31931.691688368057
31931.024555121527
31930.359385850694
31929.693543836805
31929.026453993054
31928.361241319446
31927.69497612847
31927.028949652777
31926.363248697915
31925.697319878473
31925.03146701389
31924.36535373264
31923.699837239583
31923.034635416665
31922.3696180555

In [8]:
demand_pub_oracle = pd.read_csv('./Data/Demand_LB_pred.csv', header=None)
solar_pub_oracle = pd.read_csv('./Data/Solar_LB_pred.csv', header=None)
price_pub_oracle = pd.read_csv('./Data/Price_LB_pred.csv', header=None)

In [9]:
print(list(net.parameters()))

[Parameter containing:
 0.9810
 0.9810
 0.9810
 0.9809
 0.9808
 0.9807
 0.9804
 0.9793
 0.9793
 0.9797
 0.9798
 0.9801
 0.9802
 0.9804
 0.9803
 0.9804
 0.9803
 0.9804
 0.9803
 0.9808
 0.9807
 0.9810
 0.9816
 0.9815
[torch.FloatTensor of size 24]
, Parameter containing:
 1.1798
 1.1799
 1.1800
 1.1800
 1.1800
 1.1799
 1.1781
 1.1765
 1.1767
 1.1776
 1.1777
 1.1778
 1.1779
 1.1781
 1.1783
 1.1785
 1.1785
 1.1779
 1.1770
 1.1767
 1.1763
 1.1769
 1.1779
 1.1795
[torch.FloatTensor of size 24]
]


In [10]:
def submit(name):
    url = 'http://interiit.tech:3000/upload'
    files = {'file': (name, open(name, 'rb'), '.text/csv', {'Content-Disposition': 'form-data'})}
    headers = {'Host': 'interiit.tech:3000', 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',\
               'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate',\
               'Referer': 'http://interiit.tech:3000/', 'Origin':'http://interiit.tech:3000'}
    r = requests.post(url, files=files, headers=headers)
    
def score(team_id):
    return pd.read_csv('http://interiit.tech:3000/results.csv').loc[team_id]['Score']

In [11]:
bid_quantity = ((demand_pub_oracle - solar_pub_oracle).values * net.state_dict()['w1'].numpy()).flatten()
bid_price = (price_pub_oracle.values * net.state_dict()['w2'].numpy()).flatten()
submission = pd.DataFrame([bid_price, bid_quantity])
submission.T.to_csv('23.csv', header=None, index=None)
submit('23.csv')
score(23)

319996